# Topic 9 - Convolutional Neural Networks

## Reading (or watching, as the case may be): 
(Andrew Ng Stanford Lecture)[https://www.youtube.com/watch?v=bNb2fEVKeEo]

## IC9A Implement Convolution
Implement a function that takes an arbitrary 1D input and an arbitrary 1D kernel, and outputs their convolution.  An easy way to check if your implementation is correct is to apply the kernel 
$$
[-1,0,1]
$$
to a sine function, and see if it returns (a scaled version of) cosine.  Apply a moving average kernel to the same function.

## 1 Implementing a CNN for song classification

First of all, we need a dataset.  I've grabbed one from the internet that consists of 100 songs per genre.  We'll limit ourselves to blues, metal, country, and hiphop.  I can use scipy to import .wav files and create a dataset

In [ ]:
from scipy.io import wavfile
import numpy as np
import os

genres = ['blues','metal','country','hiphop']

N=len(genres)

wavlist = []
labels = []

# blues=0,metal=1,country=2,hiphop=3
for i,genre in enumerate(genres):
    files = os.listdir('data/genres/'+genre)
    for f in files:
        filename = 'data/genres/'+genre+'/'+f
        count,data = wavfile.read(filename)
        
        # Here, I am downsampling the data by a factor of 8, and keeping only the first 65536 features,
        # roughly one minute of song
        wavlist.append(data[:2**19:8]/2**15)
        labels.append(i)
       
y = np.array(labels)
X = np.vstack(wavlist)
print(X.shape)

Now let's plot some examples of the wave-forms

In [ ]:
import matplotlib.pyplot as plt
fig,axs = plt.subplots(nrows=4,ncols=1)
fig.set_size_inches(10,10)
for ax,index in zip(axs,[0,100,200,300]):
    ax.plot(X[index,:])
    ax.set_xlim(0,10000)

plt.show()

There are definitely differences, but maybe that's just random.  What does a couple examples of metal look like?

In [ ]:
import matplotlib.pyplot as plt
fig,axs = plt.subplots(nrows=4,ncols=1)
fig.set_size_inches(10,10)
for ax,index in zip(axs,[300,301,302,303]):
    ax.plot(X[index,:])
    ax.set_xlim(0,10000)

plt.show()

Don't know.  We'll see if we can extract some characteristic features.  To do this, we'll learn filters which are then *convolved* over the signal.  

Despite having relatively few examples (100 for each genre!), we'll want to ensure that our method can generalize, so let's split our data into test and training sets.  At the same time, we'll want to reshape our arrays into what pytorch expects.

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# Training/test set split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)

X_train = X_train.reshape((*X_train.shape,1))
X_test = X_test.reshape((*X_test.shape,1))

print(X_train.shape,y_train.shape)

Now we can do our normal ritual

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)


X_train = X_train.to(torch.float32)

#Pytorch expects channels first, so reshape
X_train = X_train.reshape(-1,1,2**16)
X_test = X_test.to(torch.float32)
X_test = X_test.reshape(-1,1,2**16)
y_train = y_train.to(torch.long)
y_test = y_test.to(torch.long)

device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

from torch.utils.data import TensorDataset

training_data = TensorDataset(X_train,y_train)
test_data = TensorDataset(X_test,y_test)

batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=training_data,
                                           batch_size=batch_size, 
                                           shuffle=True)

batch_size = 64
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                           batch_size=batch_size, 
                                           shuffle=False)

Now, we can define our model.  

In [ ]:
class Net(nn.Module):
    def __init__(self):
        """
        This method is where you'll want to instantiate parameters and functions.
        Here, we instantiate four 1D convolution layers and four max pooling layers
        """
        super(Net,self).__init__()
        # Conv1d arguments: number of input feature maps, number of output feature maps, kernel width, edge padding
        self.conv1 = nn.Conv1d(1,10,9,padding=4)
        # MaxPool1d arguments: kernel width, edge padding
        self.pool1 = nn.MaxPool1d(16,padding=4)
        
        self.conv2 = nn.Conv1d(10,10,9,padding=4)
        self.pool2 = nn.MaxPool1d(16,padding=4)
        
        self.conv3 = nn.Conv1d(10,10,9,padding=4)
        self.pool3 = nn.MaxPool1d(16,padding=4)
        
        self.conv4 = nn.Conv1d(10,10,9,padding=4)
        self.pool4 = nn.MaxPool1d(16,padding=4)
        
        # Final linear transformation layer (10 features get output from final convolution, mapped to four
        # Softmax inputs)
        self.l1 = nn.Linear(10,4)
    
    def forward(self,x):
        """
        This method runs the feedforward neural network.  
        """
        
        # Apply convolution
        a1 = self.conv1(x)
        
        # Apply activation function
        z1 = torch.relu(a1)
        
        # Apply max pooling
        z1 = self.pool1(z1)
                
        a2 = self.conv2(z1)
        z2 = torch.relu(a2)
        z2 = self.pool2(z2)
        
        a3 = self.conv3(z2)
        z3 = torch.relu(a3)
        z3 = self.pool3(z3)
        
        a4 = self.conv4(z3)
        z4 = torch.relu(a4)
        z4 = self.pool4(z4)        
        
        # Flatten the array (basically removes singleton dimension)
        z_flat = torch.reshape(z4,(-1,10))
        
        # Apply linear transformation
        z_out = self.l1(z_flat)
        
        # Output logits to softmax
        return z_out

Now, our training proceeds very much as it has in the past, because our neural network is simply a function from inputs to outputs.  

In [ ]:
model = Net()
model.to(device)
criterion = torch.nn.CrossEntropyLoss(reduction='mean')

optimizer = torch.optim.Adam(model.parameters())

train_accs = []
test_accs = []

epochs = 3000
# Loop over the data
for epoch in range(epochs):
    model.train()
    # Loop over each subset of data
    for d,t in train_loader:

        # Zero out the optimizer's gradient buffer
        optimizer.zero_grad()
        
        # Make a prediction based on the model
        outputs = model(d)
        
        # Compute the loss
        loss = criterion(outputs,t)   

        # Use backpropagation to compute the derivative of the loss with respect to the parameters
        loss.backward()
        
        # Use the derivative information to update the parameters
        optimizer.step()
        
    model.eval()
    # After each epoch, compute the test set accuracy
    if epoch%10==0:
        total=0.
        correct=0.
        # Loop over all the test examples and accumulate the number of correct results in each batch
        for d,t in test_loader:
            outputs = model(d)
            _, predicted = torch.max(outputs.data,1)
            total += float(t.size(0))
            correct += float((predicted==t).sum())
        total_train = 0
        correct_train = 0
        for d,t in train_loader:
            outputs = model(d)
            _, predicted = torch.max(outputs.data,1)
            total_train += float(t.size(0))
            correct_train += float((predicted==t).sum())
        
        # Print the epoch, the training loss, and the test set accuracy.
        train_accs.append(100.*correct_train/total_train)
        test_accs.append(100.*correct/total)

        print(epoch,loss.item(),train_accs[-1],test_accs[-1])
        


Are we overfitting?

In [ ]:
plt.plot(train_accs,label='Training accuracy')
plt.plot(test_accs,label='Test accuracy')
plt.show()

Yeah, probably.  However, all of our old techniques for regularization can still apply here. 

## 2 Application to images.
While CNNs are cool for 1D sequences, their primary application has been in the field of image analysis.  Next time, we'll see how these networks provide high-grade performance on some interesting and difficult image classification tasks!

